In [3]:
import csv

def read_csv(file_name):
    with open(file_name, 'r') as f:
        reader = csv.reader(f)
        data = list(reader)
    return data

In [4]:
data = read_csv('/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/MLM/list_content_word_v2/ADV.csv')

In [5]:
import torch

def apply_eval(data):
    """
    Apply eval function to all elements in a list.
    
    Args:
    - data (list): List of strings
    
    Returns:
    - list: List with elements evaluated
    """
    # Define required functions and modules
    eval_locals = {'tensor': torch.tensor, 'torch': torch}
    
    return [eval(item, eval_locals) for item in data]

In [6]:

def convert_to_dict(data):
    list_data_dict = []
    
    for i in range(1, len(data)):
        items = apply_eval(data[i])
        f = {"uid": items[0], "word_vector": items[1].clone().detach(), "sum_vector": items[2].clone().detach(), "avg_vector": items[3].clone().detach()}
        list_data_dict.append(f)
    return list_data_dict


In [7]:
a = convert_to_dict(data)

In [8]:
len(a)

4489

In [9]:
# read json file and convert to dict
import json

import json
import torch

def read_data(readPath):

    with open(readPath, 'r', encoding = 'utf-8') as file:
        taskData = []
        for i, line in enumerate(file):
            sample = json.loads(line)
            taskData.append(sample)

    return taskData


In [10]:
file_data = read_data('/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/MLM/word_present_each_file/mlm_abolish_full.json')

# convert 2-d list to 1-d list
file_data =  [item for sublist in file_data for item in sublist]

In [11]:
print(type(file_data[0]))

<class 'dict'>


In [10]:
file_data[0].keys()

dict_keys(['input_id', 'pos_tag_id', 'word', 'sum_vector', 'avg_vector'])

## calculate cosine 

In [22]:
import torch
from sklearn.metrics.pairwise import cosine_similarity



def cosine_sim_sen_list_word(word_dict, list_dict_content_word):
    # tinh cosine trong 1 cau voi tat ca cac content word
    # word_vector: vector cua 1 tu trong file dong tu
    # list_content_word: list cac tu content word
   
    result_dict = {"sum_cosine_neg": 0, "word_neg_sum": torch.tensor([]), 
                   "sum_cosine_0": 1, "word_0_sum": torch.tensor([]), 
                   "avg_cosine_neg": 0, "word_neg_avg":  torch.tensor([]), 
                   "avg_cosine_0": 1, "word_0_avg":  torch.tensor([])}
   
    min_cosine_neg_sum = 0
    min_cosine_0_sum = 1
    min_cosine_neg_avg = 0
    min_cosine_0_avg = 1
    
    for content_word in list_dict_content_word:
       
        # SUM VECTOR 
        sum_tensor = torch.tensor(word_dict['sum_vector']).clone().detach().unsqueeze(0)
        sum_content_tensor = torch.tensor(content_word['sum_vector']).clone().detach().unsqueeze(0)
        cosine_val_sum = cosine_similarity(sum_tensor, sum_content_tensor)
        if cosine_val_sum <= min_cosine_neg_sum:
            min_cosine_neg_sum = cosine_val_sum
            result_dict['word_neg_sum'] = content_word
            
        elif cosine_val_sum <= min_cosine_0_sum:
            min_cosine_0_sum = cosine_val_sum
            result_dict['word_0_sum'] =  content_word
         
         
        # AVG VECTOR
        avg_tensor = torch.tensor(word_dict['avg_vector']).clone().detach().unsqueeze(0)
        content_word_tensor = torch.tensor(content_word['avg_vector']).clone().detach().unsqueeze(0)
        cosine_val_avg = cosine_similarity(avg_tensor, content_word_tensor)
        
        if cosine_val_avg <= min_cosine_neg_avg:
            min_cosine_neg_avg = cosine_val_avg
            result_dict['word_neg_avg'] = content_word
        elif cosine_val_avg <= min_cosine_0_avg:
            min_cosine_0_avg = cosine_val_avg
            result_dict['word_0_avg'] =  content_word
         
    result_dict['sum_cosine_neg'] = min_cosine_neg_sum  
    result_dict['sum_cosine_0'] = min_cosine_0_sum
    return result_dict
            
def create_new_data_neg_sum(predicate_file, content_word_data, wriDir):
    '''
    return 4 data cho moi file:
        sum_vector va cosine = -1: uid, origin_input_id, sum_neg_input_id, pos_tag_id
        sum_vector va cosine = 0: uid, origin_input_id, sum_0_input_id, pos_tag_id
        avg_vector va cosine = -1: uid, origin_input_id, avg_neg_input_id, pos_tag_id
        avg_vector va cosine = 0: uid, origin_input_id, avg_0_input_id, pos_tag_id
    
    '''
    predicate_data = read_data(predicate_file)
    predicate_data =  [item for sublist in predicate_data for item in sublist]
    result_dict = {}
    for i in range(len(predicate_data)):
        
        # chua xu li cai pos tag de truyen vao content_word_data cho dung tung loai tu 
        
        masked_index = torch.where(torch.tensor(predicate_data[i]['pos_tag_id']).clone().detach() != 0)
     
        if predicate_data[i]['pos_tag_id'][masked_index[0][0].item()] == 4:
            result_dict = cosine_sim_sen_list_word(predicate_data[i], content_word_data)
           
        # sentence (cau trong moi file) : dict_keys(['input_id', 'pos_tag_id', 'word', 'sum_vector', 'avg_vector'])
        # list content: uid,word,sum_vector,avg_vector
        
        # test trang tu thoi, mot if day du cho nay 
        # sum_vector va cosine = -1
            if type(result_dict['word_neg_sum']) != torch.Tensor:
                
                new_word_neg_sum = result_dict['word_neg_sum']['word_vector']
                predicate_data[i]['input_id'][masked_index[0].item()] = new_word_neg_sum
            
        
    new_data =  [{key: value for key, value in d.items() if key not in ['word_vector', 'sum_vector', 'avg_vector']} for d in predicate_data]
    # write to json file
    with open(wriDir + 'neg_sum_{}'.format(predicate_file.replace('mlm_', '')), 'w') as file:
        json.dump(new_data, file)   
        

In [23]:
file_data = '/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/MLM/word_present_each_file/mlm_abolish_full.json'
wriDir = '/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/MLM/pertured_data'
create_new_data_neg_sum(file_data, a, wriDir)


In [1]:

tuple = (1, 2, 3, 4, 5)
iterator = iter(tuple)
for i in iterator:
    print(i)

1
2
3
4
5
